# SMPLH-basic

SMPLH extends SMPL with poses of hands.

In this notebook, we will introduce the basic usage of `smplx.SMPLH` model, and some common problems you may encounter.

And there is a "SMPL family", which is a set of models related to SMPL. You can have a overview of them in [SMPL wiki](https://meshcapade.wiki/SMPL).

The basic usage of SMPLH is quite like SMPL, so we will be brief in the basic concept, and mainly focus on the differences between them. However, the **data of SMPL and SMPLH have some differences** (from the inputs to the outputs), you shouldn't mix them up.

## Preparation

You should also prepare the environments for SMPL, check the [SMPL-basic](./SMPL_basic.ipynb) for more details.

You can get the checkpoints from [MANO](https://mano.is.tue.mpg.de/), click "Download/Extended SMPL+H model" to get the checkpoint. You may notice that it has checkpoints in `.npz` postfix and checkpoints in `.pkl` postfix. The difference is that: **models with `.npz` has betas with 16 components, while models with `.pkl` have classic 10 components betas**.

After downloading the SMPLH's checkpoints, you should put them to `data_inputs/body_models/smplh`, your directory tree should look like this:

```
.
├── SMPLH_FEMALE.pkl    # beta 10
├── SMPLH_MALE.pkl      # beta 10
├── female
│   └── model.npz       # beta 16
├── male
│   └── model.npz       # beta 16
└── neutral
    └── model.npz       # beta 16
```

## Tutorials

### Environment Preparation

In [ ]:
# Packages you may use very often.
import torch
import numpy as np
from smplx import SMPL, SMPLH

# Things you don't need to care about. They are just for driving the tutorials.
from lib.path_manager import PathManager
from ez4d.vis.wis3d import HWis3D as Wis3D
from ez4d.kinematics.abstract_skeletons import Skeleton_SMPL24, Skeleton_SMPL22

pm = PathManager()

### Load SMPLH model

In [27]:
B = 150

body_models = {}
genders = ['female', 'male']  # case insensitive

for gender in genders:
    body_models[gender] = SMPLH(
            model_path = pm.inputs / 'body_models' / 'smplh',
            gender     = gender,
            batch_size = 150,
        )

# Prepare some parameters for later inference.
body_model : SMPLH = body_models['male']  # use male for example
print(body_model.shapedirs.shape)        # (6890, 3, 10)

# Prepare mesh template for later visualization.
# Tips: mesh = vertices + faces, and the faces are the indices of vertices, which won't change across SMPL's outputs.
mesh_temp : np.ndarray = body_model.faces
print(mesh_temp.shape)  # (13776, 3)

torch.Size([6890, 3, 10])
(13776, 3)


We can find that, the shape of `faces` doesn't changes. Which means the **vertices of SMPLH model is similar to SMPL model's**. We will dive into this later.

As we mentioned before, the `betas` of SMPLH model can have 16 components, although it's not directly supported by `smplx.SMPLH`, if you want to use it, you can use the following codes:

In [26]:
from typing import Union
from pathlib import Path
from smplx.utils import Struct

def build_smplh_16(bm_path:Union[str, Path], batch_size:int):
    if isinstance(bm_path, Path):
        bm_path = str(bm_path)

    smplh_dict = np.load(bm_path, encoding="latin1")
    data_struct = Struct(**smplh_dict)
    data_struct.hands_componentsl = np.zeros((0))
    data_struct.hands_componentsr = np.zeros((0))
    data_struct.hands_meanl = np.zeros((15 * 3))
    data_struct.hands_meanr = np.zeros((15 * 3))
    V, D, B = data_struct.shapedirs.shape
    data_struct.shapedirs = np.concatenate(
        [data_struct.shapedirs, np.zeros((V, D, SMPL.SHAPE_SPACE_DIM - B))],
        axis=-1,
    )  # super hacky way to let smplh use 16-size beta
    kwargs = {
            "model_type"           : "smplh",
            "data_struct"          : data_struct,
            "num_betas"            : 16,
            "num_expression_coeffs": 10,
            "use_pca"              : False,
            "flat_hand_mean"       : False,
            "batch_size"           : batch_size,
        }
    return SMPLH(bm_path, **kwargs)

body_models_16 = {}
genders_16 = ['neutral', 'female', 'male']  # case sensitive

for gender in genders_16:
    bm_path = pm.inputs / 'body_models' / 'smplh' / gender / 'model.npz'
    body_models_16[gender] = build_smplh_16(bm_path, B)

body_model_16 : SMPLH = body_models_16['neutral']
print(body_model_16.shapedirs.shape)  # (6890, 3, 16)

# Prepare mesh template for later visualization.
# Tips: mesh = vertices + faces, and the faces are the indices of vertices, which won't change across SMPL's outputs.
mesh_temp_16 : np.ndarray = body_model_16.faces
print(mesh_temp_16.shape)  # (13776, 3)

torch.Size([6890, 3, 16])
(13776, 3)


### SMPL-H Inference

In [28]:
# Inference.
smplh_out = body_model(
        betas         = torch.zeros(B, 10),
        global_orient = torch.zeros(B, 3),
        body_pose     = torch.zeros(B, 63),
        transl        = torch.zeros(B, 3),
    )

# Check output.
joints : torch.Tensor = smplh_out.joints    # (B, 73, 3)
verts  : torch.Tensor = smplh_out.vertices  # (B, 6890, 3)
print(joints.shape, verts.shape)

torch.Size([150, 73, 3]) torch.Size([150, 6890, 3])


I want to highlight that, the input format of `smplx.SMPLH` is not similar to `smplx.SMPL`'s. For example, the shape of the tensor inputted to `body_pose`, changes from `(B, J, 3)` to `(B, J'*3)`, where `J = 23` and `J' = 21`, so does the `global_orient`, which changes from `(B, 1, 3)` to `(B, 1*3)`.

> The reason why `J'` decreases to 21 is that, SMPL-X decouples the pose of hands from body pose to `left_hand_pose` and `right_hand_pose`, so that it can express more complex hand movements. Check the paper for details if you are interested. Hands are not what we are going to talk about here.

You will see later that the output mesh of SMPL and SMPL-H is quite similar but still with some small differences.

In [32]:
smplh_wis3d = Wis3D(
        pm.outputs / 'wis3d',
        'SMPLH',
    )

smplh_wis3d.add_motion_verts(
        verts  = verts[:1],
        name   = f'smplh_T_pose',
        offset = 0,
    )

In [6]:
# Start the server. (Remember to terminate the cell before going on.)
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

Serving on http://0.0.0.0:19090
^C


### SMPL-H v.s. SMPL

The output of SMPL and SMPL-H are quite alike, but there are still some small differences. For example, you will see that the hands of SMPL-H are more realistic than SMPL's. 

In [29]:
# Generate the SMPL T-pose.
body_model_smpl = SMPL(model_path=pm.inputs/'body_models'/'smpl', gender="neutral")
smpl_out = body_model_smpl()
smpl_verts = smpl_out.vertices
smpl_joints = smpl_out.joints

In [39]:
smpl_vs_smplh_wis3d = Wis3D(
        pm.outputs / 'wis3d',
        'SMPL v.s. SMPLH',
    )

# Visualize SMPL results.
smpl_vs_smplh_wis3d.add_motion_verts(
    verts  = smpl_verts[:1],
    name   = f'smpl_T_pose',
    offset = 0,
)

smpl_vs_smplh_wis3d.add_motion_skel(
    joints = smpl_joints[:1, :22],
    bones  = Skeleton_SMPL22.bones,
    colors = Skeleton_SMPL22.bone_colors,
    name   = f'smpl_T_pose',
    offset = 0,
)

# Visualize SMPL-H results.
smpl_vs_smplh_wis3d.add_motion_verts(
    verts  = verts[:1],
    name   = f'smplh_T_pose',
    offset = 0,
)

smpl_vs_smplh_wis3d.add_motion_skel(
    joints = joints[:1, :22],
    bones  = Skeleton_SMPL22.bones,
    colors = Skeleton_SMPL22.bone_colors,
    name   = f'smplh_T_pose',
    offset = 0,
)

# Use J_regressor to get the skeleton.
# If you need to change the results to SMPL's format, this way might be better.
J_regressor = body_model_smpl.J_regressor
print(J_regressor.shape)  # (24, 6890)
joints_regressed = J_regressor @ verts
print(joints_regressed.shape)  # (B, 24, 3)

smpl_vs_smplh_wis3d.add_motion_skel(
    joints = joints_regressed[:1, :22],
    bones  = Skeleton_SMPL22.bones,
    colors = Skeleton_SMPL22.bone_colors,
    name   = f'smplh_T_pose_regressed',
    offset = 0,
)

torch.Size([24, 6890])
torch.Size([150, 24, 3])


In [38]:
# Start the server. (Remember to terminate the cell before going on.)
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

Serving on http://0.0.0.0:19090
^C


## Additional Resources / Questions & Answers

1. Can I transfer the parameters between SMPL, SMPL-H and SMPL-X?
   - Yes, check the URLs below:
     - https://github.com/vchoutas/smplx/blob/main/transfer_model/README.md